### 네이버 뉴스 정보 가져오기

### 학습 내용 
 * 네이버 뉴스 여러개 단어 검색
 * 페이지별 언론사, 뉴스기사, URL

### 01. 웹 브라우저 띄우기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

### 네이버 뉴스 검색

In [2]:
driver = webdriver.Chrome('chromedriver_91')

url = 'https://search.naver.com/search.naver?query=apple&where=news'
driver.get(url)

import time
time.sleep(3)  # 홈페이지 로딩 시간 3초

In [3]:
# keyword_multi = ['이강덕 포항시장 백신', '태백시 류태호 백신', '허석 순천시장 백신', 
#                  '박영선 백신', '이강덕 시장 백신', '고3 백신', '최대호 안양시장 백신']

keyword_multi = ['백신 부작용', '백신 사망', '해외 백신', '질병관리청',
                 '백신 국민청원', '정부 백신대책']
                 
pages = 30

In [4]:
### 뉴스 검색 : //*[@id="nx_query"]
### 검색 버튼 : //*[@id="lnb.searchForm"]/fieldset/button

In [5]:
from datetime import datetime

now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)
print(now.day - 1)
now_day = now.day
now_hour = now.hour
file_make_time

19


'2021_0920_00'

In [6]:
import re
from datetime import timedelta

In [7]:
def date_cal(now_time):
    now = datetime.now()

    # 시간이 포함되어 있고,
    chk_time = now_time.find("시간")
    chk_day = now_time.find("일")
    if chk_time>=0: 
        hour = int( re.findall("\d+", now_time)[0] )
        
        # 현재 시간 - 적힌 시간이 음수가 나오면 현재 날짜에 -1을 해 준다. 
        cal_hour = now.hour - hour
        # print(cal_hour)
        if cal_hour < 0:
            news_date = timedelta(days = 1)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
        else:
            news_date = timedelta(days = 0)
            cal_now = now - news_date
            #print("현재 날짜 :",  cal_now )
            
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        print("뉴스 날짜 : ", cal_date_time)
    
    # 일이 포함되어 있고,
    elif chk_day>=0: 
        news_day = int( re.findall("\d+", now_time)[0] )
        # print(news_day)
        
        # 현재 날짜 -1일전을 계산한다.
        news_date = timedelta(days = news_day)
        cal_now = now - news_date
        
        cal_date_time = "%04d-%02d-%02d" % (cal_now.year, cal_now.month, cal_now.day)
        # print("뉴스 날짜 : ", cal_date_time)
    else:
        cal_date_time = now_time
    return cal_date_time    

In [8]:
# date_cal("3일 전")

In [9]:
def keyword_news_n(keyword1, now_time):
    now_page_num = 0
    all_company1 = []
    all_article1 = []
    all_link1 = []
    all_date1 = []
    
    for i in range(1,pages,1):
        # 페이지 부분 정보 획득
        sel_num = driver.find_element_by_class_name("sc_page_inner")
        sel_all = sel_num.find_elements_by_class_name("btn")
        
        # 화면에 보이는 페이지 확인
        print("현재 페이지 : ", now_page_num+1, "키워드 : ", keyword1, "작성시간: ", now_time)
        len_all = len(sel_all)
        
        # 페이지 번호 확인 후, 해당되는 버튼 클릭
        for i in range(1, len_all+1 ):
            tmp_pre = '//*[@id="main_pack"]/div[2]/div/div/a[' + str(i) + ']'
            sel_num = driver.find_element_by_xpath(tmp_pre)
            sel_page_num = int(sel_num.text)
            if sel_page_num == (now_page_num+1):
                # 정보를 가져와서 클릭
                # 현재 페이지 입력
                sel_num.click()
                time.sleep(1)
                now_page_num += 1 
                print(now_page_num)
                break

        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        all_ul = soup.find("ul", class_="list_news").find_all("li", class_="bx")

        for one in all_ul:
            # 언론사
            com = one.find("a", class_="info press").text
            com_a = com.replace("언론사 선정", "")

            # print(one)
            tmp = one.find("a", class_="news_tit")

            article_n = tmp.text
            tmp_link = tmp.attrs['href']
            
            # 날짜
            date_one = one.find('div', class_='info_group').find("span", class_="info").text
            now_dates = date_cal(date_one)  # 날짜 계산
            
            print(article_n, com_a, now_dates)
            print("url : ", tmp_link)

            all_company1.append(com_a)
            all_article1.append(article_n)
            all_link1.append(tmp_link)
            all_date1.append(now_dates)
            
        print(len(all_article))
    return all_company1, all_article1, all_link1, all_date1

In [10]:
import pandas as pd
from datetime import datetime
import os

### 폴더 생성

In [11]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

try:
    if not os.path.exists(file_make_time):
        os.makedirs(file_make_time)
        print("folder creating ok")
except OSError:
    print("Error : Creating directory ", file_make_time)

folder creating ok


In [12]:
now = datetime.now()
file_make_time = "%04d_%02d%02d_%02d" % (now.year, now.month, now.day, now.hour)

all_company = []
all_article = []
all_link = []
all_date = []

for one in keyword_multi:
    url = 'https://search.naver.com/search.naver?query=' + one + '&where=news'
    driver.get(url)
    time.sleep(3)
    
    sel_search = driver.find_element_by_xpath('//*[@id="nx_query"]') 
    sel_btn = driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button') 
    
    sel_search.clear()
    sel_search.send_keys(one)
    sel_btn.click()
    
    all_company, all_article, all_link, all_date = keyword_news_n(one, 
                                                                  file_make_time)
    
    dict_dat = {"언론사": all_company, "기사명" : all_article, 
                "날짜":all_date, "URL":all_link}
    
    dat = pd.DataFrame(dict_dat)
    dat['키워드'] = one
    dat.columns = ['키워드', "언론사", "기사명", "날짜", "URL"]
    
    fname_csv = file_make_time + "/" + one + "_" + file_make_time +".csv"
    fname_xlsx = file_make_time + "/" + one + "_" + file_make_time +".xlsx"
    dat.to_csv(fname_csv, index=False)
    dat.to_excel(fname_xlsx, index=False)

현재 페이지 :  1 키워드 :  백신 부작용 작성시간:  2021_0920_00
1
뉴스 날짜 :  2021-09-19
[단독]25세 여성 백신접종 후 '뇌출혈'…父 "내가 접종 강요했는데" 머니투데이 2021-09-19
url :  http://news.mt.co.kr/mtview.php?no=2021091911065282299
뉴스 날짜 :  2021-09-19
또 백신 맞고 '소장 괴사'…"건강했던 동생 살려달라" 靑 청원 매일신문 2021-09-19
url :  https://news.imaeil.com/page/view/2021091918465497771
뉴스 날짜 :  2021-09-19
백신접종 후 언제쯤? 진통제 먹었다면?…헌혈 A~Z 뉴시스 2021-09-19
url :  http://www.newsis.com/view/?id=NISX20210919_0001588952&cID=10201&pID=10200
뉴스 날짜 :  2021-09-19
간미연, 백신 부작용?…"모더나 맞고 허리 통증" [TEN★] 텐아시아 2021-09-19
url :  https://tenasia.hankyung.com/topic/article/2021091957214
“친구가 백신 맞고 성불구”...美 팝스타 주장에 백악관도 ‘발칵’ 매일경제 2021-09-18
url :  http://news.mk.co.kr/newsRead.php?no=897896&year=2021
오늘부터 백신 미접종자 사전예약 실시…10월부터 접종 뉴스1 2021-09-19
url :  https://www.news1.kr/articles/?4438805
20대 女 두명, 화이자 접종 후 뇌출혈…"백신 부작용 책임져야" 데일리안 2021-09-19
url :  https://www.dailian.co.kr/news/view/1034696/?sc=Naver
美 FDA 위원회 "화이자 부스터샷 반대…부작용·효능 대비 실익 안 커" 한국경제 2021-09-19
url :  https://www.han

현재 페이지 :  8 키워드 :  백신 부작용 작성시간:  2021_0920_00
8
[Pick] AZ→화이자 교차 접종…"며칠째 코피 안 멈춰" 부작용 호소 SBS 2021-09-17
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006468058&plink=ORI&cooper=NAVER
"백신 맞고 하혈·생리불순" 여성의 불안에 응답하라 여성신문 2021-09-18
url :  http://www.womennews.co.kr/news/articleView.html?idxno=215675
"얼굴 엄청 붓더니"...'모더나 백신' 접종 후 부작용 호소한 야옹이 작가 인사이트 2021-09-17
url :  https://www.insight.co.kr/news/358890
홍콩서 청소년 화이자 백신 부작용 속출 '심장염만 30명' 국제뉴스 2021-09-18
url :  http://www.gukjenews.com/news/articleView.html?idxno=2312072
"40대 남편, 화이자 맞고 뇌경색"…접종 부작용 국민청원 잇따라 MBN 2021.09.11.
url :  http://www.mbn.co.kr/pages/news/newsView.php?news_seq_no=4594752
[만물상] 청소년 백신 접종 조선일보 A26면 1단
url :  https://www.chosun.com/opinion/manmulsang/2021/09/16/QGYTBTM2JRBPVG4R74YNP25RIQ/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news
"코로나19 백신 부스터샷은 시기상조" 아시아경제 2면 TOP
url :  https://view.asiae.co.kr/article/2021091410315777808
전문가들 "백신 접종 후 6개월 지나면 면역력 떨어져 추가 접종 필요" 세계일보 2021-09-18
url :  http://www.

현재 페이지 :  15 키워드 :  백신 부작용 작성시간:  2021_0920_00
15
다음은 소아·청소년·임신부…백신 맞아도 될까요 머니투데이 2021-09-19
url :  http://news.mt.co.kr/mtview.php?no=2021091716260398527
홍콩, 화이자 접종 청소년들 부작용 발생에 2차 접종 중단 한국경제 2021-09-17
url :  https://www.hankyung.com/international/article/2021091618497
"백신 부작용 인정, 피해 보상금은 0원"…정부 안내문 이유는? 머니투데이 2021-09-15
url :  http://news.mt.co.kr/mtview.php?no=2021091414191931912
“화이자 맞은 父, 피 토하며 숨져…백신이 목숨 담보인 줄 몰랐다” 靑청원 ... 조선비즈 2021.09.11.
url :  https://biz.chosun.com/topics/topics_social/2021/09/11/IHRPUVJSHRERFETSCJJVNDQFOM/?utm_source=naver&utm_medium=original&utm_campaign=biz
오늘 1차 접종률 70% 넘는다…잔여백신으로 2차 접종 가능 헤럴드경제 2021-09-18
url :  http://news.heraldcorp.com/view.php?ud=20210917000229
바이든 "백신 미접종이 팬데믹" 접종 대폭 의무화 새 전략 발표(종합) 뉴시스 2021.09.10.
url :  http://www.newsis.com/view/?id=NISX20210910_0001578730&cID=10101&pID=10100
눈 다래끼 치료하러 병원 갔다 의료진 실수로 백신 맞은 초등생 세계일보 2021-09-15
url :  http://www.segye.com/content/html/2021/09/14/20210914517723.html?OutUrl=naver
佛 "1회 접종 얀센 백신으로 중증예

22
김포 20대 이어 광주 30대男, 화이자백신 1차 접종후 사망 매일경제 A31면 1단
url :  http://news.mk.co.kr/newsRead.php?no=883665&year=2021
[종합] 12~17세 코로나 백신 접종, 부작용 없을까 '대상·일정 발표는 언제?' 시사매거진 2021-09-14
url :  http://www.sisamagazine.co.kr/news/articleView.html?idxno=401281
“건강한 男청소년, 백신 위험이 이득보다 클 수 있다” 하이닥 2021-09-14
url :  http://www.hidoc.co.kr/healthstory/news/C0000633957
송도소방서, 코로나19 백신 예방접종센터 지원 FPN 2021-09-18
url :  http://fpn119.co.kr/164333
입대 앞둔 20대 초등교사 화이자 백신 접종 후 숨져 헤럴드경제 2021.09.08.
url :  http://news.heraldcorp.com/view.php?ud=20210908000655
"건강하던 40대 남편, 백신 맞고 뇌경색으로 중환자실 치료" 한국일보 2021.09.10.
url :  https://hankookilbo.com/News/Read/A2021091016130001225?did=NA
프랑스 “얀센 백신 중증 예방 실패 사례…조사 필요” KBS 2021-09-15
url :  https://news.kbs.co.kr/news/view.do?ncd=5279561&ref=A
정부, ‘#고마워 백신’ 접종 인증 캠페인 진행…“우수 소감자에 상품” 서울신문 2021-09-14
url :  https://www.seoul.co.kr/news/newsView.php?id=20210913500127&wlog_tag3=naver
화이자 "백신 효과 2개월마다 6%씩 ↓…부스터샷 필요하다" 머니투데이 2021-09-17
url :  http://news.mt.co.kr/mtview.php?no=2021

현재 페이지 :  29 키워드 :  백신 부작용 작성시간:  2021_0920_00
29
프랑스 "백신 안 맞은 보건 인력은 무급 정직"...수술 취소 등 혼란 YTN사이언스 2021-09-17
url :  https://science.ytn.co.kr/program/program_view.php?s_mcd=0082&s_hcd=&key=202109161102345026
엔지켐생명과학, KB증권에 3200억 공모…"백신 생산·공급 목적" 약사공론 2021-09-18
url :  https://www.kpanews.co.kr/article/show.asp?idx=225632&category=D
EU, 아스트라제네카 백신 부작용에 길랭-바레 증후군 추가 SBS 2021.09.09.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006459826&plink=ORI&cooper=NAVER
노바백스, 코로나·독감 동시예방 결합백신 임상시험 시작 아시아경제 2021.09.09.
url :  https://view.asiae.co.kr/article/2021090907242883788
"종잇값이 더 비싸겠다"…AZ 부작용 인정했지만 보상 안내문엔 '0원' 뉴스1 2021-09-15
url :  https://www.news1.kr/articles/?4434386
"화이자 백신 접종 2주 만에 아버지 심장이 멈췄다" 아주경제 2021.09.11.
url :  http://www.ajunews.com/view/20210911174706633
[매경데스크] 백신 안맞아도 차별 말아야 매일경제 A35면 TOP
url :  http://news.mk.co.kr/newsRead.php?no=880175&year=2021
“10대男, 화이자 부작용 확률이 코로나 입원보다 높다” MBN 2021-09-14
url :  http://mbn.mk.co.kr/pages/news/newsView.php?category=mbn00008&news

...

url :  http://www.newdaily.co.kr/site/data/html/2021/08/29/2021082900016.html
백신 이상반응 관찰 중인 시민들 뉴스1 2021.09.02.
url :  https://www.news1.kr/photos/view/?4953791
순천서 화이자 백신 맞은 20대 남 초등교사 사망 '논란' 남도일보 2021.09.08.
url :  http://www.namdonews.com/news/articleView.html?idxno=658524
백혈병, 생리불순, 의식불명…꼬리무는 백신 부작용 청원 UPI뉴스 2021.09.01.
url :  http://www.upinews.kr/newsView/upi202109010120
“여성에게 하혈은 가장 공포스러운 일”…백신 맞고 생리 이상 호소 서울신문 2021.09.01.
url :  https://www.seoul.co.kr/news/newsView.php?id=20210901500003&wlog_tag3=naver
290
현재 페이지 :  25 키워드 :  백신 국민청원 작성시간:  2021_0920_00
25
'집종 후 백혈병' 이어지는 청와대 국민청원 뉴스1 2021.09.02.
url :  https://www.news1.kr/photos/view/?4953775
백신 접종 후 월경 이상반응 사례 잇달아…당국 "인과관계 조사" 강원일보 2021.09.01.
url :  http://www.kwnews.co.kr/nview.asp?aid=221083100171
"체육 교사 예비신랑, 화이자 접종 후 급성 백혈병 진단받아" 靑 청원 세계일보 2021.08.28.
url :  http://www.segye.com/content/html/2021/08/28/20210828503986.html?OutUrl=naver
“여성에게 하혈은 가장 공포스러운 일”…백신 맞고 생리 이상 호소 서울신문 2021.09.01.
url :  https://www.seoul.co.kr

3
정부 "백신 접종자 인센티브 확대…접종 간격 단축도" 한국경제 2021-09-16
url :  https://www.hankyung.com/society/article/2021091567127
"잇단 극단적 선택에도 자영업자에게만 야박"…특단 대책 촉구 이데일리 2021-09-17
url :  http://www.edaily.co.kr/news/newspath.asp?newsid=02315686629180776
유효기간 바로 알고 백신 맞는다…백신상자에 '날짜 스티커' JTBC 2021.09.10.
url :  https://news.jtbc.joins.com/article/article.aspx?news_id=NB12023348
외국인 확진자 증가에 정부, 검사 및 백신접종 독려나서 파이낸셜뉴스 2021-09-16
url :  http://www.fnnews.com/news/202109151047359282
코로나19 백신 1차 접종, 추석 직전 70% 넘길듯 아시아경제 4면 1단
url :  https://view.asiae.co.kr/article/2021091610393170064
文대통령 "2차 백신 목표 조기 달성...가장 안전한 나라될 것" 파이낸셜뉴스 2021-09-15
url :  http://www.fnnews.com/news/202109141502442592
코로나 백신 1차 접종, 오늘 국민 70% 돌파할 듯 한국경제TV 2021-09-18
url :  http://www.wowtv.co.kr/NewsCenter/News/Read?articleId=A202109170084&t=NN
“백신 1차 접종률 70%” 말한 與…접종 완료율은 뒤에서 7등인데 조선비즈 2021-09-18
url :  https://biz.chosun.com/policy/politics/2021/09/17/RNNGRHF5QZCTXOO6BPKPH2VQ24/?utm_source=naver&utm_medium=original&utm_campaign=biz
[속보] 내일부

현재 페이지 :  10 키워드 :  정부 백신대책 작성시간:  2021_0920_00
10
백신 1차 접종률 70% '눈앞'…이르면 내일 가능 헤럴드경제 2021-09-17
url :  http://news.heraldcorp.com/view.php?ud=20210916000356
위드코로나, 백신 없이 성립 불가…전문가 “국산개발·해외도입 투트랙으로 ... 조선비즈 2021-09-17
url :  https://biz.chosun.com/it-science/bio-science/2021/09/16/UYN6WFM4F5AXTPAFEZHDT2HBX4/?utm_source=naver&utm_medium=original&utm_campaign=biz
구리에서 또 오접종...유효기간 지난 백신 105명 맞아 YTN 2021.09.08.
url :  https://www.ytn.co.kr/_ln/0103_202109082214324524
코로나 백신 접종률, OECD 최하위서 3개월만에 껑충 뉴스1 2021-09-16
url :  https://www.news1.kr/articles/?4434870
고향 갈 거면 백신 맞아라?… “접종이 원한다고 바로 되나, 황당” 세계일보 A10면 1단
url :  http://www.segye.com/content/html/2021/09/15/20210915519151.html?OutUrl=naver
'먹는' 코로나 치료제 가시화…정부 "선구매 협의 중" SBS 2021.09.09.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006460314&plink=ORI&cooper=NAVER
추석연휴 전 ‘백신 1차 접종 70%’ 예상에도…수도권 확진 비중 80% 육박 경향신문 2021-09-14
url :  https://www.khan.co.kr/national/health-welfare/article/202109131647001
오늘부터 추석 특별방역대책 시행…"이번 주 1차 접종

17
안성시, 추석 특별방역대책 추진 뉴스핌 2021-09-16
url :  http://www.newspim.com/news/view/20210915000943
'NIP 백신' 수급난 겪는 개원가 "유통사 담합 의심, 조사해야" 메디파나뉴스 2021-09-18
url :  http://medipana.com/news/news_viewer.asp?NewsNum=285215&MainKind=A&NewsKind=5&vCount=12&vKind=1
“연방정부 직원, 백신 안맞으면 징계” 바이든의 초강수 서울신문 2021.09.10.
url :  https://www.seoul.co.kr/news/newsView.php?id=20210910500087&wlog_tag3=naver
정부 "한주 앞 추석이 방역 최대고비…연휴기간 수칙 준수해야" 노컷뉴스 2021-09-13
url :  https://www.nocutnews.co.kr/news/5623492
당국 "백신 의무화 검토 안한다" 세계일보 2021.09.11.
url :  http://www.segye.com/content/html/2021/09/10/20210910512524.html?OutUrl=naver
옥천군민 절반 2차 완료 …백신 접종률 '으뜸' 충청투데이 2021-09-16
url :  https://www.cctoday.co.kr/news/articleView.html?idxno=2149567
지난주 외국인 확진자 전체 14.9%…얀센 백신 활용해 외국인 접종률↑(상보) 아시아경제 2021-09-16
url :  https://view.asiae.co.kr/article/2021091511534885894
美백신접종 의무화…공화당 '소송' 정치화 노컷뉴스 2021-09-14
url :  https://www.nocutnews.co.kr/news/5623546
내년 9000만회분 추가 구매?… 당장 맞을 백신도 없는데 조선비즈 2021.09.01.
url :  https://biz.chosun.co

24
정부 "백신 인센티브 확대·방역수칙 단순화 검토" 세계일보 A2면 1단
url :  http://www.segye.com/content/html/2021/09/10/20210910514001.html?OutUrl=naver
백신 1차 접종률서 美·日 앞질렀다…추석 전 전국민 70% 넘을 듯 세계일보 2021-09-13
url :  http://www.segye.com/content/html/2021/09/12/20210912503988.html?OutUrl=naver
세종·충남·충북 거리노숙인 백신 접종률 0%… 방역 허점 될라 충청투데이 2021-09-17
url :  https://www.cctoday.co.kr/news/articleView.html?idxno=2149699
美의료 전문가들 "백신 의무화 조치, 장기적으로 확산세 막을 수 있어" 뉴스1 2021.09.11.
url :  https://www.news1.kr/articles/?4431489
백신 1차 접종률 70% 돌파···다음 목표는 '10월말 접종완료 70%' 팩트경제신문 2021-09-18
url :  https://www.facten.co.kr/news/articleView.html?idxno=206855
오늘부터 추석 특별방역대책 시행…요양시설 접촉면회 가능 뉴스1 2021-09-14
url :  https://www.news1.kr/articles/?4432161
모더나 백신, 오늘 오후 126.3만 회분 도착…약속 물량 못 지켜 SBS 2021.09.05.
url :  https://news.sbs.co.kr/news/endPage.do?news_id=N1006454382&plink=ORI&cooper=NAVER
"韓 대중음악 공연산업 죽어간다" 음공협, 대책 마련 호소[전문] 스타뉴스 2021.09.08.
url :  http://star.mt.co.kr/stview.php?no=2021090814432979027
외국인 확진 늘어 15% 달해..."백신 접종 당부" YT